In [ ]:
import tensorflow as tf
from keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Reshape
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

X_train = tf.image.resize(X_train, [32, 32])
X_test = tf.image.resize(X_test, [32, 32])

X_train = tf.concat([X_train, X_train, X_train], axis=-1)
X_test = tf.concat([X_test, X_test, X_test], axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

In [ ]:
x = base_model.output
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from keras.callbacks import EarlyStopping
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 32
epochs = 10

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience = 1,
    restore_best_weights = True
)

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping])

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
import keras
model.save('NN/mnist_densenet.h5')